## Identify outliers

Which genes can not be predicted?

In [12]:
import os
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

# sklearn imports
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

In [13]:
data = pd.read_csv("results_data/mdl_data.csv")
# tidy the data
data['m6A'] = data.m6A.astype(np.float)

data.head()

,Gene_ID,time,is_maternal,ARE,MiR_4302nd6mer,utr_log,m6A,MiR430,PLS1,PLS2,wt_polya,wt_ribo
0,ENSDARG00000000018,6,True,2,1,7.205635,0.0,0,0.866097,1.623199,-0.610867,-1.255142
1,ENSDARG00000000019,6,True,2,1,7.643962,0.0,2,1.076742,0.877583,-1.253481,-0.691014
2,ENSDARG00000000068,6,True,1,0,6.608001,0.0,0,2.158753,1.800105,-0.560254,-0.715011
3,ENSDARG00000000086,6,True,1,0,5.720312,0.0,0,1.392857,0.715777,-1.437790,-0.495296
4,ENSDARG00000000103,6,True,1,0,7.144407,0.0,0,2.140356,0.917946,0.126062,-0.299922


## Initial Split (Test Data)

In [14]:
data_train, data_test = train_test_split(data, test_size=.10, random_state=42)
predictor_vars = ['m6A', 'MiR430', 'PLS1', 'PLS2', 'ARE', 'MiR_4302nd6mer', 'utr_log']

X_train = data_train[predictor_vars]
X_test = data_train[predictor_vars]
y_train_ribo = data_train.wt_ribo
y_train_polya = data_train.wt_polya
X_train.head()

,m6A,MiR430,PLS1,PLS2,ARE,MiR_4302nd6mer,utr_log
2527,0.0,0,1.790076,-1.191588,0,0,3.988984
2856,1.0,0,0.993364,0.268633,1,0,4.770685
2700,0.0,0,-0.839153,-2.207092,1,0,6.045005
149,0.0,0,-0.243841,0.614113,0,0,6.493754
3678,0.0,0,-1.870393,0.762896,0,0,5.590987


## Data Pre-processing

I will only use centering and scaling. There are 6 features for this model:

1. PLS1 (optimality)
2. PLS2 (optimality)
3. M6A (m6A target?)
4. miR-430 (# 6-mer sites)

## Tain SVM model

For hyperparamter tunning I will use randomnized search.

In [15]:
regressor = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', SVR(kernel='rbf', gamma=0.1))
]) 

grid = param_grid={"regressor__C": [0.001, 0.01, 0.1, 1e0, 1e1, 1e2, 1e3], "regressor__gamma": np.logspace(-2, 2, 5)}

# train model in poly-A data
svr_poly = GridSearchCV(regressor, param_grid=grid, scoring='r2', cv=10, verbose=1, n_jobs=31)
svr_poly.fit(X_train, y_train_polya)

Fitting 10 folds for each of 35 candidates, totalling 350 fits


[Parallel(n_jobs=31)]: Done 138 tasks      | elapsed:   12.5s
[Parallel(n_jobs=31)]: Done 350 out of 350 | elapsed:  4.0min finished


GridSearchCV(cv=10, error_score='raise',
       estimator=Pipeline(steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('regressor', SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False))]),
       fit_params={}, iid=True, n_jobs=31,
       param_grid={'regressor__gamma': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02]), 'regressor__C': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='r2', verbose=1)

In [16]:
# train model in ribo data
svr_ribo = GridSearchCV(regressor, param_grid=grid, scoring='r2', cv=10, verbose=1, n_jobs=31)
svr_ribo.fit(X_train, y_train_ribo)

Fitting 10 folds for each of 35 candidates, totalling 350 fits


[Parallel(n_jobs=31)]: Done 138 tasks      | elapsed:   12.5s
[Parallel(n_jobs=31)]: Done 350 out of 350 | elapsed:  4.2min finished


GridSearchCV(cv=10, error_score='raise',
       estimator=Pipeline(steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('regressor', SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False))]),
       fit_params={}, iid=True, n_jobs=31,
       param_grid={'regressor__gamma': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02]), 'regressor__C': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='r2', verbose=1)

In [18]:
svr_ribo.best_score_

0.17957718620126562

In [19]:
svr_poly.best_score_

0.1978127463180188

## Generate Predictions

In [20]:
# test data

data['predicitions_polyA'] = svr_poly.predict(data[predictor_vars])
data['predictions_ribo'] = svr_ribo.predict(data[predictor_vars])

In [21]:
data.to_csv('results_data/predictions_results.csv', index=False)